In [1]:
import pandas as pd  
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
csv = '../datasets/combined/data/all_tweets/num_label_tweets.csv'
my_df = pd.read_csv(csv)#,index_col=0)
my_df.head()

,text,target
0,people will give panamaverdict in na election ...,1
1,just barking dog where did panama expose corru...,1
2,it is clear from maryam nawaz bitter na campai...,1
3,nab references triggered by panama verdict are...,1
4,jaise motu gang saath hua tha he did not accep...,1


In [3]:
my_df.dropna(inplace=True)
my_df.reset_index(drop=True,inplace=True)
my_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 965236 entries, 0 to 965235
Data columns (total 2 columns):
text      965236 non-null object
target    965236 non-null int64
dtypes: int64(1), object(1)
memory usage: 14.7+ MB


In [4]:
x = my_df.text
y1 = my_df.target

In [5]:
from sklearn.cross_validation import train_test_split
SEED = 2000
x_train, x_validation_and_test, y_train1, y_validation_and_test1 = train_test_split(x, y1, test_size=.02, random_state=SEED)
x_validation, x_test, y_validation1, y_test1 = train_test_split(x_validation_and_test, y_validation_and_test1, test_size=.5, random_state=SEED)

/home/alizeb/.local/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# Neural Networks with Doc2Vec

In [7]:
from gensim.models import Doc2Vec

def get_concat_vectors(model1,model2, corpus, size):
    vecs = np.zeros((len(corpus), size))
    n = 0
    for i in corpus.index:
        prefix = 'all_' + str(i)
        vecs[n] = np.append(model1.docvecs[prefix],model2.docvecs[prefix])
        n += 1
    return vecs

In [8]:
model_ug_dbow = Doc2Vec.load('../datasets/combined/models/classification/d2v_model_ug_dbow.doc2vec')
model_tg_dmm = Doc2Vec.load('../datasets/combined/models/classification/d2v_model_tg_dmm.doc2vec')
model_ug_dbow.delete_temporary_training_data(keep_doctags_vectors=True, keep_inference=True)
model_tg_dmm.delete_temporary_training_data(keep_doctags_vectors=True, keep_inference=True)

/home/alizeb/.local/lib/python3.5/site-packages/gensim/models/doc2vec.py:362: UserWarning: The parameter `iter` is deprecated, will be removed in 4.0.0, use `epochs` instead.
  warnings.warn("The parameter `iter` is deprecated, will be removed in 4.0.0, use `epochs` instead.")
/home/alizeb/.local/lib/python3.5/site-packages/gensim/models/doc2vec.py:366: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


In [9]:
train_vecs_ugdbow_tgdmm = get_concat_vectors(model_ug_dbow,model_tg_dmm, x_train, 200)
validation_vecs_ugdbow_tgdmm = get_concat_vectors(model_ug_dbow,model_tg_dmm, x_validation, 200)

In [9]:
%%time
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()
clf.fit(train_vecs_ugdbow_tgdmm, y_train1)

CPU times: user 3min 44s, sys: 532 ms, total: 3min 45s
Wall time: 3min 45s


In [10]:
%%time
clf.score(train_vecs_ugdbow_tgdmm, y_train1)

CPU times: user 512 ms, sys: 8 ms, total: 520 ms
Wall time: 540 ms


0.9831520480880741

In [11]:
%%time
clf.score(validation_vecs_ugdbow_tgdmm, y_validation1)

CPU times: user 12 ms, sys: 0 ns, total: 12 ms
Wall time: 8.1 ms


0.9818690426854538

In [59]:
import keras
import csv
nb_classes =6
x = []
y_train = []

with open('../datasets/combined/data/all_tweets/num_label_tweets.csv', 'r') as train_file:
    reader = csv.reader(train_file)
    next(reader, None)
    for row in reader:
        text = row[0]
        target = row[1]
        
        x.append(text)
        y_train.append(target)
        
y = keras.utils.to_categorical(y_train, nb_classes)

In [61]:
from sklearn.cross_validation import train_test_split
SEED = 2000
x_train, x_validation_and_test, y_train, y_validation_and_test = train_test_split(x, y, test_size=.02, random_state=SEED)
x_validation, x_test, y_validation, y_test = train_test_split(x_validation_and_test, y_validation_and_test, test_size=.5, random_state=SEED)

In [57]:
seed = 7

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

In [18]:
print(model_d2v_01.summary())
from keras.models import model_from_json
# serialize model to JSON
model_json = model_d2v_01.to_json()
with open("../datasets/combined/models/classification/keras_model_d2v_01.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model_d2v_01.save_weights("../datasets/combined/models/classification/keras_model_d2v_01.h5")
print("Saved model to disk")

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 64)                12864     
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 390       
Total params: 13,254
Trainable params: 13,254
Non-trainable params: 0
_________________________________________________________________
None
Saved model to disk


In [15]:
np.random.seed(seed)
model_d2v_02 = Sequential()
model_d2v_02.add(Dense(64, activation='relu', input_dim=200))
model_d2v_02.add(Dense(64, activation='relu'))
model_d2v_02.add(Dense(nb_classes, activation='sigmoid'))
model_d2v_02.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model_d2v_02.fit(train_vecs_ugdbow_tgdmm, y_train, validation_data=(validation_vecs_ugdbow_tgdmm, y_validation), epochs=10, batch_size=32, verbose=2)

Train on 945931 samples, validate on 9652 samples
Epoch 1/10
84s - loss: 0.0147 - acc: 0.9949 - val_loss: 0.0114 - val_acc: 0.9957
Epoch 2/10
82s - loss: 0.0095 - acc: 0.9967 - val_loss: 0.0106 - val_acc: 0.9961
Epoch 3/10
82s - loss: 0.0085 - acc: 0.9970 - val_loss: 0.0098 - val_acc: 0.9967
Epoch 4/10
82s - loss: 0.0079 - acc: 0.9972 - val_loss: 0.0092 - val_acc: 0.9965
Epoch 5/10
82s - loss: 0.0075 - acc: 0.9973 - val_loss: 0.0094 - val_acc: 0.9967
Epoch 6/10
82s - loss: 0.0071 - acc: 0.9975 - val_loss: 0.0093 - val_acc: 0.9965
Epoch 7/10
82s - loss: 0.0068 - acc: 0.9975 - val_loss: 0.0107 - val_acc: 0.9962
Epoch 8/10
82s - loss: 0.0066 - acc: 0.9976 - val_loss: 0.0098 - val_acc: 0.9964
Epoch 9/10
82s - loss: 0.0064 - acc: 0.9977 - val_loss: 0.0100 - val_acc: 0.9968
Epoch 10/10
82s - loss: 0.0062 - acc: 0.9977 - val_loss: 0.0102 - val_acc: 0.9959


In [19]:
print(model_d2v_02.summary())
from keras.models import model_from_json
# serialize model to JSON
model_json = model_d2v_02.to_json()
with open("../datasets/combined/models/classification/keras_model_d2v_02.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model_d2v_02.save_weights("../datasets/combined/models/classification/keras_model_d2v_02.h5")
print("Saved model to disk")

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 64)                12864     
_________________________________________________________________
dense_4 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_5 (Dense)              (None, 6)                 390       
Total params: 17,414
Trainable params: 17,414
Non-trainable params: 0
_________________________________________________________________
None
Saved model to disk


In [17]:
%%time
np.random.seed(seed)
model_d2v_03 = Sequential()
model_d2v_03.add(Dense(64, activation='relu', input_dim=200))
model_d2v_03.add(Dense(64, activation='relu'))
model_d2v_03.add(Dense(64, activation='relu'))
model_d2v_03.add(Dense(nb_classes, activation='sigmoid'))
model_d2v_03.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model_d2v_03.fit(train_vecs_ugdbow_tgdmm, y_train, validation_data=(validation_vecs_ugdbow_tgdmm, y_validation), epochs=10, batch_size=32, verbose=2)

Train on 945931 samples, validate on 9652 samples
Epoch 1/10
96s - loss: 0.0150 - acc: 0.9948 - val_loss: 0.0113 - val_acc: 0.9956
Epoch 2/10
95s - loss: 0.0096 - acc: 0.9967 - val_loss: 0.0098 - val_acc: 0.9965
Epoch 3/10
92s - loss: 0.0086 - acc: 0.9970 - val_loss: 0.0097 - val_acc: 0.9967
Epoch 4/10
92s - loss: 0.0080 - acc: 0.9972 - val_loss: 0.0098 - val_acc: 0.9965
Epoch 5/10
92s - loss: 0.0076 - acc: 0.9973 - val_loss: 0.0122 - val_acc: 0.9963
Epoch 6/10
92s - loss: 0.0073 - acc: 0.9974 - val_loss: 0.0110 - val_acc: 0.9962
Epoch 7/10
92s - loss: 0.0071 - acc: 0.9975 - val_loss: 0.0102 - val_acc: 0.9967
Epoch 8/10
93s - loss: 0.0069 - acc: 0.9975 - val_loss: 0.0113 - val_acc: 0.9966
Epoch 9/10
92s - loss: 0.0067 - acc: 0.9976 - val_loss: 0.0118 - val_acc: 0.9964
Epoch 10/10
97s - loss: 0.0067 - acc: 0.9976 - val_loss: 0.0130 - val_acc: 0.9964
CPU times: user 26min 39s, sys: 2min 8s, total: 28min 47s
Wall time: 15min 35s


In [20]:
print(model_d2v_03.summary())
from keras.models import model_from_json
# serialize model to JSON
model_json = model_d2v_03.to_json()
with open("../datasets/combined/models/classification/keras_model_d2v_03.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model_d2v_03.save_weights("../datasets/combined/models/classification/keras_model_d2v_03.h5")
print("Saved model to disk")

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 64)                12864     
_________________________________________________________________
dense_7 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_8 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_9 (Dense)              (None, 6)                 390       
Total params: 21,574
Trainable params: 21,574
Non-trainable params: 0
_________________________________________________________________
None
Saved model to disk


In [21]:
%%time
np.random.seed(seed)
model_d2v_04 = Sequential()
model_d2v_04.add(Dense(128, activation='relu', input_dim=200))
model_d2v_04.add(Dense(nb_classes, activation='sigmoid'))
model_d2v_04.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model_d2v_04.fit(train_vecs_ugdbow_tgdmm, y_train, validation_data=(validation_vecs_ugdbow_tgdmm, y_validation), epochs=10, batch_size=32, verbose=2)

Train on 945931 samples, validate on 9652 samples
Epoch 1/10
74s - loss: 0.0149 - acc: 0.9950 - val_loss: 0.0111 - val_acc: 0.9958
Epoch 2/10
76s - loss: 0.0094 - acc: 0.9968 - val_loss: 0.0096 - val_acc: 0.9964
Epoch 3/10
75s - loss: 0.0085 - acc: 0.9971 - val_loss: 0.0104 - val_acc: 0.9962
Epoch 4/10
74s - loss: 0.0079 - acc: 0.9973 - val_loss: 0.0100 - val_acc: 0.9965
Epoch 5/10
76s - loss: 0.0074 - acc: 0.9975 - val_loss: 0.0100 - val_acc: 0.9966
Epoch 6/10
74s - loss: 0.0070 - acc: 0.9976 - val_loss: 0.0104 - val_acc: 0.9964
Epoch 7/10
75s - loss: 0.0066 - acc: 0.9977 - val_loss: 0.0111 - val_acc: 0.9962
Epoch 8/10
75s - loss: 0.0063 - acc: 0.9978 - val_loss: 0.0107 - val_acc: 0.9966
Epoch 9/10
73s - loss: 0.0061 - acc: 0.9979 - val_loss: 0.0118 - val_acc: 0.9961
Epoch 10/10
78s - loss: 0.0058 - acc: 0.9980 - val_loss: 0.0123 - val_acc: 0.9961
CPU times: user 23min 14s, sys: 2min 4s, total: 25min 18s
Wall time: 12min 37s


In [22]:
print(model_d2v_04.summary())
from keras.models import model_from_json
# serialize model to JSON
model_json = model_d2v_04.to_json()
with open("../datasets/combined/models/classification/keras_model_d2v_04.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model_d2v_04.save_weights("../datasets/combined/models/classification/keras_model_d2v_04.h5")
print("Saved model to disk")

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 128)               25728     
_________________________________________________________________
dense_11 (Dense)             (None, 6)                 774       
Total params: 26,502
Trainable params: 26,502
Non-trainable params: 0
_________________________________________________________________
None
Saved model to disk


In [23]:
%%time
np.random.seed(seed)
model_d2v_05 = Sequential()
model_d2v_05.add(Dense(128, activation='relu', input_dim=200))
model_d2v_05.add(Dense(128, activation='relu'))
model_d2v_05.add(Dense(nb_classes, activation='sigmoid'))
model_d2v_05.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model_d2v_05.fit(train_vecs_ugdbow_tgdmm, y_train, validation_data=(validation_vecs_ugdbow_tgdmm, y_validation), epochs=10, batch_size=32, verbose=2)

Train on 945931 samples, validate on 9652 samples
Epoch 1/10
85s - loss: 0.0136 - acc: 0.9953 - val_loss: 0.0117 - val_acc: 0.9958
Epoch 2/10
87s - loss: 0.0091 - acc: 0.9968 - val_loss: 0.0101 - val_acc: 0.9961
Epoch 3/10
84s - loss: 0.0080 - acc: 0.9972 - val_loss: 0.0092 - val_acc: 0.9970
Epoch 4/10
84s - loss: 0.0072 - acc: 0.9974 - val_loss: 0.0097 - val_acc: 0.9965
Epoch 5/10
84s - loss: 0.0066 - acc: 0.9977 - val_loss: 0.0098 - val_acc: 0.9967
Epoch 6/10
84s - loss: 0.0061 - acc: 0.9978 - val_loss: 0.0101 - val_acc: 0.9965
Epoch 7/10
84s - loss: 0.0058 - acc: 0.9979 - val_loss: 0.0114 - val_acc: 0.9963
Epoch 8/10
84s - loss: 0.0054 - acc: 0.9980 - val_loss: 0.0126 - val_acc: 0.9963
Epoch 9/10
84s - loss: 0.0052 - acc: 0.9981 - val_loss: 0.0117 - val_acc: 0.9965
Epoch 10/10
84s - loss: 0.0050 - acc: 0.9982 - val_loss: 0.0128 - val_acc: 0.9965
CPU times: user 24min 51s, sys: 2min 12s, total: 27min 3s
Wall time: 14min 7s


In [24]:
print(model_d2v_05.summary())
from keras.models import model_from_json
# serialize model to JSON
model_json = model_d2v_05.to_json()
with open("../datasets/combined/models/classification/keras_model_d2v_05.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model_d2v_05.save_weights("../datasets/combined/models/classification/keras_model_d2v_05.h5")
print("Saved model to disk")

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 128)               25728     
_________________________________________________________________
dense_13 (Dense)             (None, 128)               16512     
_________________________________________________________________
dense_14 (Dense)             (None, 6)                 774       
Total params: 43,014
Trainable params: 43,014
Non-trainable params: 0
_________________________________________________________________
None
Saved model to disk


In [25]:
%%time
np.random.seed(seed)
model_d2v_06 = Sequential()
model_d2v_06.add(Dense(128, activation='relu', input_dim=200))
model_d2v_06.add(Dense(128, activation='relu'))
model_d2v_06.add(Dense(128, activation='relu'))
model_d2v_06.add(Dense(nb_classes, activation='sigmoid'))
model_d2v_06.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model_d2v_06.fit(train_vecs_ugdbow_tgdmm, y_train, validation_data=(validation_vecs_ugdbow_tgdmm, y_validation), epochs=10, batch_size=32, verbose=2)

Train on 945931 samples, validate on 9652 samples
Epoch 1/10
94s - loss: 0.0142 - acc: 0.9951 - val_loss: 0.0114 - val_acc: 0.9958
Epoch 2/10
94s - loss: 0.0094 - acc: 0.9968 - val_loss: 0.0093 - val_acc: 0.9963
Epoch 3/10
94s - loss: 0.0083 - acc: 0.9971 - val_loss: 0.0087 - val_acc: 0.9968
Epoch 4/10
94s - loss: 0.0076 - acc: 0.9973 - val_loss: 0.0124 - val_acc: 0.9960
Epoch 5/10
94s - loss: 0.0071 - acc: 0.9975 - val_loss: 0.0102 - val_acc: 0.9965
Epoch 6/10
94s - loss: 0.0067 - acc: 0.9976 - val_loss: 0.0099 - val_acc: 0.9964
Epoch 7/10
97s - loss: 0.0064 - acc: 0.9977 - val_loss: 0.0097 - val_acc: 0.9968
Epoch 8/10
94s - loss: 0.0061 - acc: 0.9978 - val_loss: 0.0117 - val_acc: 0.9966
Epoch 9/10
95s - loss: 0.0058 - acc: 0.9979 - val_loss: 0.0127 - val_acc: 0.9964
Epoch 10/10
94s - loss: 0.0057 - acc: 0.9979 - val_loss: 0.0124 - val_acc: 0.9964
CPU times: user 26min 46s, sys: 2min 15s, total: 29min 1s
Wall time: 15min 48s


In [26]:
print(model_d2v_06.summary())
from keras.models import model_from_json
# serialize model to JSON
model_json = model_d2v_06.to_json()
with open("../datasets/combined/models/classification/keras_model_d2v_06.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model_d2v_06.save_weights("../datasets/combined/models/classification/keras_model_d2v_06.h5")
print("Saved model to disk")

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_15 (Dense)             (None, 128)               25728     
_________________________________________________________________
dense_16 (Dense)             (None, 128)               16512     
_________________________________________________________________
dense_17 (Dense)             (None, 128)               16512     
_________________________________________________________________
dense_18 (Dense)             (None, 6)                 774       
Total params: 59,526
Trainable params: 59,526
Non-trainable params: 0
_________________________________________________________________
None
Saved model to disk


In [27]:
%%time
np.random.seed(seed)
model_d2v_07 = Sequential()
model_d2v_07.add(Dense(256, activation='relu', input_dim=200))
model_d2v_07.add(Dense(nb_classes, activation='sigmoid'))
model_d2v_07.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model_d2v_07.fit(train_vecs_ugdbow_tgdmm, y_train, validation_data=(validation_vecs_ugdbow_tgdmm, y_validation), epochs=10, batch_size=32, verbose=2)

Train on 945931 samples, validate on 9652 samples
Epoch 1/10
75s - loss: 0.0139 - acc: 0.9953 - val_loss: 0.0109 - val_acc: 0.9960
Epoch 2/10
75s - loss: 0.0092 - acc: 0.9969 - val_loss: 0.0097 - val_acc: 0.9966
Epoch 3/10
75s - loss: 0.0081 - acc: 0.9972 - val_loss: 0.0104 - val_acc: 0.9963
Epoch 4/10
78s - loss: 0.0073 - acc: 0.9975 - val_loss: 0.0099 - val_acc: 0.9965
Epoch 5/10
75s - loss: 0.0066 - acc: 0.9977 - val_loss: 0.0105 - val_acc: 0.9966
Epoch 6/10
75s - loss: 0.0060 - acc: 0.9979 - val_loss: 0.0111 - val_acc: 0.9969
Epoch 7/10
75s - loss: 0.0055 - acc: 0.9981 - val_loss: 0.0129 - val_acc: 0.9962
Epoch 8/10
75s - loss: 0.0051 - acc: 0.9982 - val_loss: 0.0126 - val_acc: 0.9963
Epoch 9/10
75s - loss: 0.0047 - acc: 0.9983 - val_loss: 0.0146 - val_acc: 0.9963
Epoch 10/10
75s - loss: 0.0045 - acc: 0.9984 - val_loss: 0.0145 - val_acc: 0.9962
CPU times: user 23min 10s, sys: 2min 11s, total: 25min 21s
Wall time: 12min 39s


In [28]:
print(model_d2v_07.summary())
from keras.models import model_from_json
# serialize model to JSON
model_json = model_d2v_07.to_json()
with open("../datasets/combined/models/classification/keras_model_d2v_07.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model_d2v_07.save_weights("../datasets/combined/models/classification/keras_model_d2v_07.h5")
print("Saved model to disk")

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_19 (Dense)             (None, 256)               51456     
_________________________________________________________________
dense_20 (Dense)             (None, 6)                 1542      
Total params: 52,998
Trainable params: 52,998
Non-trainable params: 0
_________________________________________________________________
None
Saved model to disk


In [29]:
%%time
np.random.seed(seed)
model_d2v_08 = Sequential()
model_d2v_08.add(Dense(256, activation='relu', input_dim=200))
model_d2v_08.add(Dense(256, activation='relu'))
model_d2v_08.add(Dense(nb_classes, activation='sigmoid'))
model_d2v_08.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model_d2v_08.fit(train_vecs_ugdbow_tgdmm, y_train, validation_data=(validation_vecs_ugdbow_tgdmm, y_validation), epochs=10, batch_size=32, verbose=2)

Train on 945931 samples, validate on 9652 samples
Epoch 1/10
91s - loss: 0.0132 - acc: 0.9954 - val_loss: 0.0110 - val_acc: 0.9959
Epoch 2/10
93s - loss: 0.0090 - acc: 0.9969 - val_loss: 0.0094 - val_acc: 0.9964
Epoch 3/10
91s - loss: 0.0078 - acc: 0.9973 - val_loss: 0.0098 - val_acc: 0.9965
Epoch 4/10
94s - loss: 0.0068 - acc: 0.9976 - val_loss: 0.0097 - val_acc: 0.9967
Epoch 5/10
98s - loss: 0.0061 - acc: 0.9978 - val_loss: 0.0105 - val_acc: 0.9963
Epoch 6/10
94s - loss: 0.0057 - acc: 0.9980 - val_loss: 0.0109 - val_acc: 0.9963
Epoch 7/10
94s - loss: 0.0051 - acc: 0.9981 - val_loss: 0.0110 - val_acc: 0.9967
Epoch 8/10
95s - loss: 0.0048 - acc: 0.9982 - val_loss: 0.0128 - val_acc: 0.9963
Epoch 9/10
94s - loss: 0.0046 - acc: 0.9983 - val_loss: 0.0131 - val_acc: 0.9964
Epoch 10/10
91s - loss: 0.0043 - acc: 0.9984 - val_loss: 0.0132 - val_acc: 0.9967
CPU times: user 25min 53s, sys: 2min 43s, total: 28min 37s
Wall time: 15min 40s


In [30]:
print(model_d2v_08.summary())
from keras.models import model_from_json
# serialize model to JSON
model_json = model_d2v_08.to_json()
with open("../datasets/combined/models/classification/keras_model_d2v_08.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model_d2v_08.save_weights("../datasets/combined/models/classification/keras_model_d2v_08.h5")
print("Saved model to disk")

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_21 (Dense)             (None, 256)               51456     
_________________________________________________________________
dense_22 (Dense)             (None, 256)               65792     
_________________________________________________________________
dense_23 (Dense)             (None, 6)                 1542      
Total params: 118,790
Trainable params: 118,790
Non-trainable params: 0
_________________________________________________________________
None
Saved model to disk


In [31]:
%%time
np.random.seed(seed)
model_d2v_09 = Sequential()
model_d2v_09.add(Dense(256, activation='relu', input_dim=200))
model_d2v_09.add(Dense(256, activation='relu'))
model_d2v_09.add(Dense(256, activation='relu'))
model_d2v_09.add(Dense(nb_classes, activation='sigmoid'))
model_d2v_09.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model_d2v_09.fit(train_vecs_ugdbow_tgdmm, y_train, validation_data=(validation_vecs_ugdbow_tgdmm, y_validation), epochs=10, batch_size=32, verbose=2)

Train on 945931 samples, validate on 9652 samples
Epoch 1/10
106s - loss: 0.0138 - acc: 0.9953 - val_loss: 0.0114 - val_acc: 0.9960
Epoch 2/10
106s - loss: 0.0094 - acc: 0.9968 - val_loss: 0.0104 - val_acc: 0.9961
Epoch 3/10
107s - loss: 0.0081 - acc: 0.9972 - val_loss: 0.0095 - val_acc: 0.9968
Epoch 4/10
107s - loss: 0.0072 - acc: 0.9975 - val_loss: 0.0100 - val_acc: 0.9965
Epoch 5/10
107s - loss: 0.0065 - acc: 0.9977 - val_loss: 0.0110 - val_acc: 0.9965
Epoch 6/10
107s - loss: 0.0060 - acc: 0.9979 - val_loss: 0.0109 - val_acc: 0.9962
Epoch 7/10
107s - loss: 0.0055 - acc: 0.9980 - val_loss: 0.0097 - val_acc: 0.9969
Epoch 8/10
107s - loss: 0.0053 - acc: 0.9981 - val_loss: 0.0097 - val_acc: 0.9969
Epoch 9/10
106s - loss: 0.0049 - acc: 0.9982 - val_loss: 0.0117 - val_acc: 0.9965
Epoch 10/10
107s - loss: 0.0047 - acc: 0.9983 - val_loss: 0.0116 - val_acc: 0.9970
CPU times: user 27min 45s, sys: 3min 7s, total: 30min 53s
Wall time: 17min 50s


In [32]:
print(model_d2v_09.summary())
from keras.models import model_from_json
# serialize model to JSON
model_json = model_d2v_09.to_json()
with open("../datasets/combined/models/classification/keras_model_d2v_09.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model_d2v_09.save_weights("../datasets/combined/models/classification/keras_model_d2v_09.h5")
print("Saved model to disk")

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_24 (Dense)             (None, 256)               51456     
_________________________________________________________________
dense_25 (Dense)             (None, 256)               65792     
_________________________________________________________________
dense_26 (Dense)             (None, 256)               65792     
_________________________________________________________________
dense_27 (Dense)             (None, 6)                 1542      
Total params: 184,582
Trainable params: 184,582
Non-trainable params: 0
_________________________________________________________________
None
Saved model to disk


In [33]:
%%time
np.random.seed(seed)
model_d2v_10 = Sequential()
model_d2v_10.add(Dense(512, activation='relu', input_dim=200))
model_d2v_10.add(Dense(nb_classes, activation='sigmoid'))
model_d2v_10.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model_d2v_10.fit(train_vecs_ugdbow_tgdmm, y_train, validation_data=(validation_vecs_ugdbow_tgdmm, y_validation), epochs=10, batch_size=32, verbose=2)

Train on 945931 samples, validate on 9652 samples
Epoch 1/10
82s - loss: 0.0134 - acc: 0.9955 - val_loss: 0.0111 - val_acc: 0.9959
Epoch 2/10
80s - loss: 0.0091 - acc: 0.9969 - val_loss: 0.0096 - val_acc: 0.9965
Epoch 3/10
81s - loss: 0.0079 - acc: 0.9973 - val_loss: 0.0110 - val_acc: 0.9960
Epoch 4/10
81s - loss: 0.0068 - acc: 0.9977 - val_loss: 0.0101 - val_acc: 0.9965
Epoch 5/10
80s - loss: 0.0059 - acc: 0.9980 - val_loss: 0.0110 - val_acc: 0.9964
Epoch 6/10
81s - loss: 0.0052 - acc: 0.9982 - val_loss: 0.0113 - val_acc: 0.9967
Epoch 7/10
80s - loss: 0.0046 - acc: 0.9984 - val_loss: 0.0132 - val_acc: 0.9959
Epoch 8/10
81s - loss: 0.0041 - acc: 0.9985 - val_loss: 0.0136 - val_acc: 0.9963
Epoch 9/10
81s - loss: 0.0038 - acc: 0.9987 - val_loss: 0.0151 - val_acc: 0.9962
Epoch 10/10
81s - loss: 0.0035 - acc: 0.9988 - val_loss: 0.0176 - val_acc: 0.9958
CPU times: user 23min 42s, sys: 2min 35s, total: 26min 17s
Wall time: 13min 31s


In [34]:
print(model_d2v_10.summary())
from keras.models import model_from_json
# serialize model to JSON
model_json = model_d2v_10.to_json()
with open("../datasets/combined/models/classification/keras_model_d2v_10.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model_d2v_10.save_weights("../datasets/combined/models/classification/keras_model_d2v_10.h5")
print("Saved model to disk")

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_28 (Dense)             (None, 512)               102912    
_________________________________________________________________
dense_29 (Dense)             (None, 6)                 3078      
Total params: 105,990
Trainable params: 105,990
Non-trainable params: 0
_________________________________________________________________
None
Saved model to disk


In [35]:
%%time
np.random.seed(seed)
model_d2v_11 = Sequential()
model_d2v_11.add(Dense(512, activation='relu', input_dim=200))
model_d2v_11.add(Dense(512, activation='relu'))
model_d2v_11.add(Dense(nb_classes, activation='sigmoid'))
model_d2v_11.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model_d2v_11.fit(train_vecs_ugdbow_tgdmm, y_train, validation_data=(validation_vecs_ugdbow_tgdmm, y_validation), epochs=10, batch_size=32, verbose=2)

Train on 945931 samples, validate on 9652 samples
Epoch 1/10
116s - loss: 0.0132 - acc: 0.9955 - val_loss: 0.0112 - val_acc: 0.9961
Epoch 2/10
112s - loss: 0.0091 - acc: 0.9969 - val_loss: 0.0098 - val_acc: 0.9964
Epoch 3/10
113s - loss: 0.0078 - acc: 0.9973 - val_loss: 0.0102 - val_acc: 0.9965
Epoch 4/10
113s - loss: 0.0067 - acc: 0.9977 - val_loss: 0.0103 - val_acc: 0.9968
Epoch 5/10
113s - loss: 0.0059 - acc: 0.9979 - val_loss: 0.0116 - val_acc: 0.9964
Epoch 6/10
112s - loss: 0.0053 - acc: 0.9981 - val_loss: 0.0114 - val_acc: 0.9964
Epoch 7/10
112s - loss: 0.0047 - acc: 0.9983 - val_loss: 0.0142 - val_acc: 0.9962
Epoch 8/10
112s - loss: 0.0044 - acc: 0.9984 - val_loss: 0.0150 - val_acc: 0.9964
Epoch 9/10
112s - loss: 0.0041 - acc: 0.9985 - val_loss: 0.0158 - val_acc: 0.9964
Epoch 10/10
112s - loss: 0.0039 - acc: 0.9986 - val_loss: 0.0159 - val_acc: 0.9966
CPU times: user 27min 31s, sys: 4min 8s, total: 31min 40s
Wall time: 18min 52s


In [36]:
print(model_d2v_11.summary())
from keras.models import model_from_json
# serialize model to JSON
model_json = model_d2v_11.to_json()
with open("../datasets/combined/models/classification/keras_model_d2v_11.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model_d2v_11.save_weights("../datasets/combined/models/classification/keras_model_d2v_11.h5")
print("Saved model to disk")

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_30 (Dense)             (None, 512)               102912    
_________________________________________________________________
dense_31 (Dense)             (None, 512)               262656    
_________________________________________________________________
dense_32 (Dense)             (None, 6)                 3078      
Total params: 368,646
Trainable params: 368,646
Non-trainable params: 0
_________________________________________________________________
None
Saved model to disk


In [37]:
%%time
np.random.seed(seed)
model_d2v_12 = Sequential()
model_d2v_12.add(Dense(512, activation='relu', input_dim=200))
model_d2v_12.add(Dense(512, activation='relu'))
model_d2v_12.add(Dense(512, activation='relu'))
model_d2v_12.add(Dense(nb_classes, activation='sigmoid'))
model_d2v_12.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model_d2v_12.fit(train_vecs_ugdbow_tgdmm, y_train, validation_data=(validation_vecs_ugdbow_tgdmm, y_validation), epochs=10, batch_size=32, verbose=2)

Train on 945931 samples, validate on 9652 samples
Epoch 1/10
136s - loss: 0.0140 - acc: 0.9954 - val_loss: 0.0120 - val_acc: 0.9958
Epoch 2/10
135s - loss: 0.0094 - acc: 0.9969 - val_loss: 0.0097 - val_acc: 0.9962
Epoch 3/10
136s - loss: 0.0080 - acc: 0.9973 - val_loss: 0.0116 - val_acc: 0.9962
Epoch 4/10
136s - loss: 0.0070 - acc: 0.9975 - val_loss: 0.0105 - val_acc: 0.9963
Epoch 5/10
136s - loss: 0.0061 - acc: 0.9978 - val_loss: 0.0119 - val_acc: 0.9965
Epoch 6/10
136s - loss: 0.0056 - acc: 0.9980 - val_loss: 0.0115 - val_acc: 0.9965
Epoch 7/10
136s - loss: 0.0053 - acc: 0.9981 - val_loss: 0.0126 - val_acc: 0.9968
Epoch 8/10
136s - loss: 0.0049 - acc: 0.9983 - val_loss: 0.0143 - val_acc: 0.9966
Epoch 9/10
136s - loss: 0.0047 - acc: 0.9984 - val_loss: 0.0135 - val_acc: 0.9966
Epoch 10/10
140s - loss: 0.0046 - acc: 0.9984 - val_loss: 0.0138 - val_acc: 0.9967
CPU times: user 30min 40s, sys: 5min 6s, total: 35min 47s
Wall time: 22min 46s


In [38]:
print(model_d2v_12.summary())
from keras.models import model_from_json
# serialize model to JSON
model_json = model_d2v_12.to_json()
with open("../datasets/combined/models/classification/keras_model_d2v_12.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model_d2v_12.save_weights("../datasets/combined/models/classification/keras_model_d2v_12.h5")
print("Saved model to disk")

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_33 (Dense)             (None, 512)               102912    
_________________________________________________________________
dense_34 (Dense)             (None, 512)               262656    
_________________________________________________________________
dense_35 (Dense)             (None, 512)               262656    
_________________________________________________________________
dense_36 (Dense)             (None, 6)                 3078      
Total params: 631,302
Trainable params: 631,302
Non-trainable params: 0
_________________________________________________________________
None
Saved model to disk


| model | input layer (nodes) | hidden layer (nodes) | output layer (nodes) | best validation accuracy | number of epochs for best validation accuracy |
|-------|--------------|--------------|------------------|--------|--------|
| model_d2v_01 | 1 (200)  | 1 (64) relu  |  1 (1) sigmoid   | 99.68% | epoch 7 |
| model_d2v_02 | 1 (200)  | 2 (64) relu  |  1 (1) sigmoid   | 99.68% | epoch 9 |
| model_d2v_03 | 1 (200)  | 3 (64) relu  |  1 (1) sigmoid   | 99.67% | epoch 3 |
| model_d2v_04 | 1 (200)  | 1 (128) relu  |  1 (1) sigmoid   | 99.66% | epoch 5  |
| model_d2v_05 | 1 (200)  | 2 (128) relu  |  1 (1) sigmoid   | 99.70% | epoch 3  |
| model_d2v_06 | 1 (200)  | 3 (128) relu  |  1 (1) sigmoid   | 99.68% | epoch 7  |
| model_d2v_07 | 1 (200)  | 1 (256) relu  |  1 (1) sigmoid   | 99.69% | epoch 6  |
| model_d2v_08 | 1 (200)  | 2 (256) relu  |  1 (1) sigmoid   | 99.67% | epoch 10  |
| model_d2v_09 | 1 (200)  | 3 (256) relu |  1 (1) sigmoid   | 99.70% | epoch 10  |
| model_d2v_10 | 1 (200)  | 1 (512) relu |  1 (1) sigmoid   | 99.67% | epoch 6  |
| model_d2v_11 | 1 (200)  | 2 (512) relu |  1 (1) sigmoid   | 99.68% | epoch 4  |
| model_d2v_12 | 1 (200)  | 3 (512) relu |  1 (1) sigmoid   | 99.68% | epoch 7  |

In [39]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

filepath="../datasets/combined/models/classification/d2v_09_best_weights.{epoch:02d}-{val_acc:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
early_stop = EarlyStopping(monitor='val_acc', patience=5, mode='max') 
callbacks_list = [checkpoint, early_stop]
np.random.seed(seed)
model_d2v_09_es = Sequential()
model_d2v_09_es.add(Dense(256, activation='relu', input_dim=200))
model_d2v_09_es.add(Dense(256, activation='relu'))
model_d2v_09_es.add(Dense(256, activation='relu'))
model_d2v_09_es.add(Dense(nb_classes, activation='sigmoid'))
model_d2v_09_es.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model_d2v_09_es.fit(train_vecs_ugdbow_tgdmm, y_train, validation_data=(validation_vecs_ugdbow_tgdmm, y_validation), 
                    epochs=100, batch_size=32, verbose=2, callbacks=callbacks_list)

Train on 945931 samples, validate on 9652 samples
Epoch 1/100
Epoch 00000: val_acc improved from -inf to 0.99599, saving model to ../datasets/combined/models/classification/d2v_09_best_weights.00-0.9960.hdf5
107s - loss: 0.0138 - acc: 0.9953 - val_loss: 0.0109 - val_acc: 0.9960
Epoch 2/100
Epoch 00001: val_acc improved from 0.99599 to 0.99636, saving model to ../datasets/combined/models/classification/d2v_09_best_weights.01-0.9964.hdf5
107s - loss: 0.0094 - acc: 0.9968 - val_loss: 0.0096 - val_acc: 0.9964
Epoch 3/100
Epoch 00002: val_acc improved from 0.99636 to 0.99646, saving model to ../datasets/combined/models/classification/d2v_09_best_weights.02-0.9965.hdf5
107s - loss: 0.0081 - acc: 0.9972 - val_loss: 0.0099 - val_acc: 0.9965
Epoch 4/100
Epoch 00003: val_acc did not improve
107s - loss: 0.0072 - acc: 0.9975 - val_loss: 0.0100 - val_acc: 0.9964
Epoch 5/100
Epoch 00004: val_acc improved from 0.99646 to 0.99663, saving model to ../datasets/combined/models/classification/d2v_09_best

If I evaluate the model I just run, it will give me the result as same as I got from the last epoch.

In [40]:
model_d2v_09_es.evaluate(x=validation_vecs_ugdbow_tgdmm, y=y_validation)

8736/9652 [==========================>...] - ETA: 0s

[0.01781654689798304, 0.9962356783174846]

In [19]:
from keras.models import load_model
loaded_model = load_model('../datasets/combined/models/classification/d2v_09_best_weights.07-0.9967.hdf5')

In [20]:
loaded_model.evaluate(x=validation_vecs_ugdbow_tgdmm, y=y_validation)

9376/9652 [============================>.] - ETA: 0s

[0.011632945165446261, 0.9967364375639693]

# Word2Vec

## Word vectors extracted from Doc2Vec models (Average/Sum)

In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import scale

In [14]:
model_ug_dmm = Doc2Vec.load('../datasets/combined/models/classification/d2v_model_ug_dmm.doc2vec')
model_ug_dmm.delete_temporary_training_data(keep_doctags_vectors=True, keep_inference=True)

/home/alizeb/.local/lib/python3.5/site-packages/gensim/models/doc2vec.py:362: UserWarning: The parameter `iter` is deprecated, will be removed in 4.0.0, use `epochs` instead.
  warnings.warn("The parameter `iter` is deprecated, will be removed in 4.0.0, use `epochs` instead.")
/home/alizeb/.local/lib/python3.5/site-packages/gensim/models/doc2vec.py:366: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


In [15]:
def get_w2v_ugdbowdmm(tweet, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in tweet.split():
        try:
            vec += np.append(model_ug_dbow[word],model_ug_dmm[word]).reshape((1, size))
            count += 1.
        except KeyError:
            continue
    if count != 0:
        vec /= count
    return vec

In [16]:
def get_w2v_ugdbowdmm_sum(tweet, size):
    vec = np.zeros(size).reshape((1, size))
    for word in tweet.split():
        try:
            vec += np.append(model_ug_dbow[word],model_ug_dmm[word]).reshape((1, size))
        except KeyError:
            continue
    return vec

In [17]:
train_vecs_w2v_dbowdmm = np.concatenate([get_w2v_ugdbowdmm(z, 200) for z in x_train])
validation_vecs_w2v_dbowdmm = np.concatenate([get_w2v_ugdbowdmm(z, 200) for z in x_validation])

In [26]:
%%time
clf = LogisticRegression()
clf.fit(train_vecs_w2v_dbowdmm, y_train1)

CPU times: user 7min 59s, sys: 4.66 s, total: 8min 4s
Wall time: 7min 54s


In [27]:
clf.score(validation_vecs_w2v_dbowdmm, y_validation1)

0.9725445503522586

In [18]:
train_vecs_w2v_dbowdmm_s = scale(train_vecs_w2v_dbowdmm)
validation_vecs_w2v_dbowdmm_s = scale(validation_vecs_w2v_dbowdmm)

Let's see how summed word vectors perform compared to the averaged counter part.

In [21]:
train_vecs_w2v_dbowdmm_sum = np.concatenate([get_w2v_ugdbowdmm_sum(z, 200) for z in x_train])
validation_vecs_w2v_dbowdmm_sum = np.concatenate([get_w2v_ugdbowdmm_sum(z, 200) for z in x_validation])

In [24]:
train_vecs_w2v_dbowdmm_sum_s = scale(train_vecs_w2v_dbowdmm_sum)
validation_vecs_w2v_dbowdmm_sum_s = scale(validation_vecs_w2v_dbowdmm_sum)

In [22]:
%%time
clf = LogisticRegression()
clf.fit(train_vecs_w2v_dbowdmm_sum_s, y_train1)

CPU times: user 2min 2s, sys: 48.4 s, total: 2min 51s
Wall time: 3min 41s


In [23]:
clf.score(validation_vecs_w2v_dbowdmm_sum_s, y_validation1)

0.725250626566416

## Word vectors extracted from Doc2Vec models with TFIDF weighting (Average/Sum)

In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer
tvec = TfidfVectorizer(min_df=2)
tvec.fit_transform(x_train)
tfidf = dict(zip(tvec.get_feature_names(), tvec.idf_))
print('vocab size :', len(tfidf))

/home/alizeb/.local/lib/python3.5/site-packages/sklearn/feature_extraction/text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


vocab size : 108244


In [32]:
len(set(model_ug_dbow.wv.vocab.keys()) & set(tvec.get_feature_names()))

108244

In [42]:
def get_w2v_general(tweet, size, vectors, aggregation='mean'):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in tweet.split():
        try:
            vec += vectors[word].reshape((1, size))
            count += 1.
        except KeyError:
            continue
    if aggregation == 'mean':
        if count != 0:
            vec /= count
        return vec
    elif aggregation == 'sum':
        return vec

In [37]:
%%time
w2v_tfidf = {}
for w in model_ug_dbow.wv.vocab.keys():
    if w in tvec.get_feature_names():
        w2v_tfidf[w] = np.append(model_ug_dbow[w],model_ug_dmm[w]) * tfidf[w]

CPU times: user 3h 32min 8s, sys: 51.3 s, total: 3h 32min 59s
Wall time: 3h 31min 19s


In [39]:
import pickle
with open('../datasets/combined/models/classification/w2v_tfidf.p', 'wb') as fp:
    pickle.dump(w2v_tfidf, fp, protocol=pickle.HIGHEST_PROTOCOL)

In [25]:
import pickle
with open('../datasets/combined/models/classification/w2v_tfidf.p', 'rb') as fp:
    w2v_tfidf = pickle.load(fp)

In [30]:
%%time
train_vecs_w2v_tfidf_mean = scale(np.concatenate([get_w2v_general(z, 200, w2v_tfidf, 'mean') for z in x_train]))
validation_vecs_w2v_tfidf_mean = scale(np.concatenate([get_w2v_general(z, 200, w2v_tfidf, 'mean') for z in x_validation]))

CPU times: user 1min 2s, sys: 3.71 s, total: 1min 6s
Wall time: 3min 28s


In [49]:
%%time
clf = LogisticRegression()
clf.fit(train_vecs_w2v_tfidf_mean, y_train1)

CPU times: user 4min 24s, sys: 3.23 s, total: 4min 27s
Wall time: 4min 21s


In [50]:
clf.score(validation_vecs_w2v_tfidf_mean, y_validation1)

0.9504765851636966

In [31]:
%%time
train_vecs_w2v_tfidf_sum = scale(np.concatenate([get_w2v_general(z, 200, w2v_tfidf, 'sum') for z in x_train]))
validation_vecs_w2v_tfidf_sum = scale(np.concatenate([get_w2v_general(z, 200, w2v_tfidf, 'sum') for z in x_validation]))

CPU times: user 1min 1s, sys: 3.64 s, total: 1min 4s
Wall time: 2min 19s


In [52]:
%%time
clf = LogisticRegression()
clf.fit(train_vecs_w2v_tfidf_sum, y_train1)

CPU times: user 5min 12s, sys: 3.9 s, total: 5min 16s
Wall time: 5min 10s


In [53]:
clf.score(validation_vecs_w2v_tfidf_sum, y_validation1)

0.9487152921674264

## Word vectors extracted from Doc2Vec models with custom weighting (Average/Sum)

In [33]:
from sklearn.feature_extraction.text import CountVectorizer
cvec = CountVectorizer(max_features=100000)
cvec.fit(x_train)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=100000, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [34]:
verdict_train = x_train[y_train1 == 1]
panama_train = x_train[y_train1 == 2]
loadshedding_train = x_train[y_train1 == 3]
cpec_train = x_train[y_train1 == 4]
dengue_train = x_train[y_train1 == 5]
panama_doc_matrix = cvec.transform(panama_train)
verdict_doc_matrix = cvec.transform(verdict_train)
loadshedding_doc_matrix = cvec.transform(loadshedding_train)
cpec_doc_matrix = cvec.transform(cpec_train)
dengue_doc_matrix = cvec.transform(dengue_train)

In [35]:
%%time
panama_tf = np.sum(panama_doc_matrix,axis=0)
verdict_tf = np.sum(verdict_doc_matrix,axis=0)
loadshedding_tf = np.sum(loadshedding_doc_matrix,axis=0)
cpec_tf = np.sum(cpec_doc_matrix,axis=0)
dengue_tf = np.sum(dengue_doc_matrix,axis=0)

CPU times: user 56 ms, sys: 0 ns, total: 56 ms
Wall time: 53.8 ms


In [36]:
from scipy.stats import hmean
from scipy.stats import norm
def normcdf(x):
    return norm.cdf(x, x.mean(), x.std())

panama = np.squeeze(np.asarray(panama_tf))
verdict = np.squeeze(np.asarray(verdict_tf))
loadshedding = np.squeeze(np.asarray(loadshedding_tf))
cpec = np.squeeze(np.asarray(cpec_tf))
dengue = np.squeeze(np.asarray(dengue_tf))

term_freq_df2 = pd.DataFrame([verdict,panama,loadshedding,cpec,dengue],columns=cvec.get_feature_names()).transpose()
term_freq_df2.columns = ['verdict', 'panama', 'loadshedding', 'cpec','dengue']
term_freq_df2['total'] = term_freq_df2['verdict'] + term_freq_df2['panama'] + term_freq_df2['loadshedding'] + term_freq_df2['cpec'] + term_freq_df2['dengue']
term_freq_df2['panama_rate'] = term_freq_df2['panama'] * 1./term_freq_df2['total']
term_freq_df2['panama_freq_pct'] = term_freq_df2['panama'] * 1./term_freq_df2['panama'].sum()
term_freq_df2['panama_rate_normcdf'] = normcdf(term_freq_df2['panama_rate'])
term_freq_df2['panama_freq_pct_normcdf'] = normcdf(term_freq_df2['panama_freq_pct'])
term_freq_df2['panama_normcdf_hmean'] = hmean([term_freq_df2['panama_rate_normcdf'], term_freq_df2['panama_freq_pct_normcdf']])
term_freq_df2.sort_values(by='panama_normcdf_hmean', ascending=False).iloc[:10]

,verdict,panama,loadshedding,cpec,dengue,total,panama_rate,panama_freq_pct,panama_rate_normcdf,panama_freq_pct_normcdf,panama_normcdf_hmean
livenews,0,11556,0,0,0,11556,1.000000,0.004013,0.981120,1.000000,0.990470
liveleak,0,21577,0,0,1,21578,0.999954,0.007492,0.981114,1.000000,0.990467
fonseca,1,3835,0,1,0,3837,0.999479,0.001332,0.981054,0.999996,0.990434
mossack,1,3820,0,1,0,3822,0.999477,0.001326,0.981053,0.999996,0.990434
cdnpoli,2,3714,0,1,1,3718,0.998924,0.001290,0.980982,0.999993,0.990397
iceland,0,7543,1,8,1,7553,0.998676,0.002619,0.980950,1.000000,0.990384
clinton,0,3632,2,10,3,3647,0.995887,0.001261,0.980588,0.999989,0.990194
adult,0,6688,13,0,19,6720,0.995238,0.002322,0.980503,1.000000,0.990156
offshore,7,13046,12,42,4,13111,0.995042,0.004530,0.980477,1.000000,0.990142
panamapapers,617,129682,21,77,0,130397,0.994517,0.045029,0.980408,1.000000,0.990107


In [38]:
term_freq_df2.to_csv("../datasets/combined/data/term_freq_df2_panama.csv")

In [37]:
panama_hmean = term_freq_df2.panama_normcdf_hmean

In [61]:
%%time
w2v_panama_hmean = {}
for w in model_ug_dbow.wv.vocab.keys():
    if w in panama_hmean.keys():
        w2v_panama_hmean[w] = np.append(model_ug_dbow[w],model_ug_dmm[w]) * panama_hmean[w]

CPU times: user 3.57 s, sys: 52 ms, total: 3.62 s
Wall time: 3.59 s


In [64]:
with open('../datasets/combined/models/classification/w2v_hmean.p', 'wb') as fp:
    pickle.dump(w2v_panama_hmean, fp, protocol=pickle.HIGHEST_PROTOCOL)

In [30]:
import pickle
with open('../datasets/combined/models/classification/w2v_hmean.p', 'rb') as fp:
    w2v_panama_hmean = pickle.load(fp)

In [32]:
train_vecs_w2v_panamahmean_mean = scale(np.concatenate([get_w2v_general(z, 200, w2v_panama_hmean, 'mean') for z in x_train]))
validation_vecs_w2v_panamahmean_mean = scale(np.concatenate([get_w2v_general(z, 200, w2v_panama_hmean, 'mean') for z in x_validation]))

In [69]:
%%time
clf = LogisticRegression()
clf.fit(train_vecs_w2v_panamahmean_mean, y_train1)

CPU times: user 7min 20s, sys: 5.06 s, total: 7min 25s
Wall time: 7min 16s


In [70]:
clf.score(validation_vecs_w2v_panamahmean_mean, y_validation1)

0.9872565271446332

In [33]:
train_vecs_w2v_panamahmean_sum = scale(np.concatenate([get_w2v_general(z, 200, w2v_panama_hmean, 'sum') for z in x_train]))
validation_vecs_w2v_panamahmean_sum = scale(np.concatenate([get_w2v_general(z, 200, w2v_panama_hmean, 'sum') for z in x_validation]))

In [72]:
%%time
clf = LogisticRegression()
clf.fit(train_vecs_w2v_panamahmean_sum, y_train1)

CPU times: user 8min 3s, sys: 5.39 s, total: 8min 8s
Wall time: 8min


In [73]:
clf.score(validation_vecs_w2v_panamahmean_sum, y_validation1)

0.9881889763779528

## Word vectors extracted from pre-trained GloVe (Average/Sum)

In [36]:
#import gensim.downloader as api
glove_twitter = {}
GLOVE_TWITTER27B_200D_PATH = "../datasets/combined/data/glove.twitter.27B.200d.txt"

In [37]:
with open(GLOVE_TWITTER27B_200D_PATH, "rb") as infile:
    for line in infile:
        parts = line.split()
        word = parts[0].decode(encoding='utf8')
        #if (word in all_words):
        nums=np.array(parts[1:], dtype=np.float32)
        glove_twitter[word] = nums

In [38]:
train_vecs_glove_mean = scale(np.concatenate([get_w2v_general(z, 200, glove_twitter,'mean') for z in x_train]))
validation_vecs_glove_mean = scale(np.concatenate([get_w2v_general(z, 200, glove_twitter,'mean') for z in x_validation]))

In [39]:
%%time
clf = LogisticRegression()
clf.fit(train_vecs_glove_mean, y_train1)

CPU times: user 7min 31s, sys: 316 ms, total: 7min 31s
Wall time: 7min 36s


In [40]:
clf.score(validation_vecs_glove_mean, y_validation1)

0.9251968503937008

In [41]:
train_vecs_glove_sum = scale(np.concatenate([get_w2v_general(z, 200, glove_twitter,'sum') for z in x_train]))
validation_vecs_glove_sum = scale(np.concatenate([get_w2v_general(z, 200, glove_twitter,'sum') for z in x_validation]))

In [42]:
%%time
clf = LogisticRegression()
clf.fit(train_vecs_glove_sum, y_train1)

CPU times: user 7min 58s, sys: 420 ms, total: 7min 58s
Wall time: 8min


In [43]:
clf.score(validation_vecs_glove_sum, y_validation1)

0.9292374637380854

## Word vectors extracted from pre-trained Google News Word2Vec (Average/Sum)

In [17]:
train_vecs_googlenews_mean = scale(np.concatenate([get_w2v_general(z, 300, googlenews,'mean') for z in x_train]))
validation_vecs_googlenews_mean = scale(np.concatenate([get_w2v_general(z, 300, googlenews,'mean') for z in x_validation]))

In [18]:
%%time
clf = LogisticRegression()
clf.fit(train_vecs_googlenews_mean, y_train)

CPU times: user 5min 55s, sys: 41min 5s, total: 47min
Wall time: 1h 23min 5s


In [19]:
clf.score(validation_vecs_googlenews_mean, y_validation)

0.749561403508772

In [20]:
train_vecs_googlenews_sum = scale(np.concatenate([get_w2v_general(z, 300, googlenews,'sum') for z in x_train]))
validation_vecs_googlenews_sum = scale(np.concatenate([get_w2v_general(z, 300, googlenews,'sum') for z in x_validation]))

In [21]:
%%time
clf = LogisticRegression()
clf.fit(train_vecs_googlenews_sum, y_train)

CPU times: user 5min 45s, sys: 39min 17s, total: 45min 2s
Wall time: 1h 19min 51s


In [22]:
clf.score(validation_vecs_googlenews_sum, y_validation)

0.7491854636591478

## Separately trained Word2Vec (Average/Sum)

In [7]:
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
import gensim
from gensim.models.word2vec import Word2Vec
from gensim.models.doc2vec import TaggedDocument
import multiprocessing
from sklearn import utils

In [8]:
def labelize_tweets_ug(tweets,label):
    result = []
    prefix = label
    for i, t in zip(tweets.index, tweets):
        result.append(TaggedDocument(t.split(), [prefix + '_%s' % i]))
    return result

In [51]:
all_x = pd.concat([x_train,x_validation,x_test])
all_x_w2v = labelize_tweets_ug(all_x, 'all')

In [53]:
import pickle
with open('../datasets/combined/models/classification/all_x_w2v.p', 'wb') as fp:
    pickle.dump(all_x_w2v, fp, protocol=pickle.HIGHEST_PROTOCOL)

In [6]:
import pickle
with open('../datasets/combined/models/classification/all_x_w2v.p', 'rb') as fp:
    all_x_w2v = pickle.load(fp)

In [55]:
%%time
cores = multiprocessing.cpu_count()
model_ug_cbow = Word2Vec(sg=0, size=100, negative=5, window=2, min_count=2, workers=cores, alpha=0.065, min_alpha=0.065)
model_ug_cbow.build_vocab([x.words for x in tqdm(all_x_w2v)])

100%|██████████| 965236/965236 [00:00<00:00, 2080003.17it/s]


CPU times: user 6.38 s, sys: 24 ms, total: 6.4 s
Wall time: 8.94 s


In [56]:
%%time
for epoch in range(30):
    model_ug_cbow.train(utils.shuffle([x.words for x in tqdm(all_x_w2v)]), total_examples=len(all_x_w2v), epochs=1)
    model_ug_cbow.alpha -= 0.002
    model_ug_cbow.min_alpha = model_ug_cbow.alpha

100%|██████████| 965236/965236 [00:00<00:00, 2505900.79it/s]


CPU times: user 12min 30s, sys: 3.17 s, total: 12min 33s
Wall time: 4min 42s


In [57]:
%%time
train_vecs_cbow_mean = scale(np.concatenate([get_w2v_general(z, 100, model_ug_cbow,'mean') for z in x_train]))
validation_vecs_cbow_mean = scale(np.concatenate([get_w2v_general(z, 100, model_ug_cbow,'mean') for z in x_validation]))

/home/alizeb/.local/lib/python3.5/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


CPU times: user 2min 57s, sys: 320 ms, total: 2min 57s
Wall time: 2min 57s


In [58]:
%%time
clf = LogisticRegression()
clf.fit(train_vecs_cbow_mean, y_train1)

CPU times: user 4min 12s, sys: 28 ms, total: 4min 12s
Wall time: 4min 13s


In [59]:
clf.score(validation_vecs_cbow_mean, y_validation1)

0.9814546208039785

In [16]:
#model_ug_cbow.save('../datasets/combined/models/classification/d2v_model_ug_cbow.word2vec')
model_ug_cbow = Word2Vec.load('../datasets/combined/models/classification/d2v_model_ug_cbow.word2vec')
#model_ug_cbow.delete_temporary_training_data(keep_doctags_vectors=True, keep_inference=True)

In [62]:
model_ug_sg = Word2Vec(sg=1, size=100, negative=5, window=2, min_count=2, workers=cores, alpha=0.065, min_alpha=0.065)
model_ug_sg.build_vocab([x.words for x in tqdm(all_x_w2v)])

100%|██████████| 965236/965236 [00:00<00:00, 2138060.94it/s]


In [63]:
%%time
for epoch in range(30):
    model_ug_sg.train(utils.shuffle([x.words for x in tqdm(all_x_w2v)]), total_examples=len(all_x_w2v), epochs=1)
    model_ug_sg.alpha -= 0.002
    model_ug_sg.min_alpha = model_ug_sg.alpha

100%|██████████| 965236/965236 [00:00<00:00, 2489479.24it/s]


CPU times: user 24min 21s, sys: 3.76 s, total: 24min 25s
Wall time: 5min 46s


In [64]:
train_vecs_sg_mean = scale(np.concatenate([get_w2v_general(z, 100, model_ug_sg,'mean') for z in x_train]))
validation_vecs_sg_mean = scale(np.concatenate([get_w2v_general(z, 100, model_ug_sg,'mean') for z in x_validation]))

/home/alizeb/.local/lib/python3.5/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [65]:
%%time
clf = LogisticRegression()
clf.fit(train_vecs_sg_mean, y_train1)

CPU times: user 2min 52s, sys: 60 ms, total: 2min 52s
Wall time: 2min 54s


In [66]:
clf.score(validation_vecs_sg_mean, y_validation1)

0.9814546208039785

In [10]:
#model_ug_sg.save('../datasets/combined/models/classification/d2v_model_ug_sg.word2vec')
model_ug_sg = Word2Vec.load('../datasets/combined/models/classification/d2v_model_ug_sg.word2vec')
#model_ug_sg.delete_temporary_training_data(keep_doctags_vectors=True, keep_inference=True)

In [41]:
def get_w2v_mean(tweet, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in tweet.split():
        try:
            vec += np.append(model_ug_cbow[word],model_ug_sg[word]).reshape((1, size))
            count += 1.
        except KeyError:
            continue
    if count != 0:
        vec /= count
    return vec

In [17]:
%%time
train_vecs_cbowsg_mean = scale(np.concatenate([get_w2v_mean(z, 200) for z in x_train]))
validation_vecs_cbowsg_mean = scale(np.concatenate([get_w2v_mean(z, 200) for z in x_validation]))

/home/alizeb/.local/lib/python3.5/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


CPU times: user 5min 59s, sys: 6.28 s, total: 6min 5s
Wall time: 6min 2s


In [18]:
%%time
clf = LogisticRegression()
clf.fit(train_vecs_cbowsg_mean, y_train1)

CPU times: user 10min 53s, sys: 496 ms, total: 10min 54s
Wall time: 10min 55s


In [19]:
import pickle
with open('../datasets/combined/models/classification/clf_lr_cbowsg_mean.p', 'wb') as fp:
    pickle.dump(clf, fp, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
import pickle
with open('../datasets/combined/models/classification/clf_lr_cbowsg_mean.p', 'rb') as fp:
    clf = pickle.load(fp)

In [20]:
clf.score(validation_vecs_cbowsg_mean, y_validation1)

0.9871529216742644

In [21]:
def get_w2v_sum(tweet, size):
    vec = np.zeros(size).reshape((1, size))
    for word in tweet.split():
        try:
            vec += np.append(model_ug_cbow[word],model_ug_sg[word]).reshape((1, size))
        except KeyError:
            continue
    return vec

In [22]:
train_vecs_cbowsg_sum = scale(np.concatenate([get_w2v_sum(z, 200) for z in x_train]))
validation_vecs_cbowsg_sum = scale(np.concatenate([get_w2v_sum(z, 200) for z in x_validation]))

/home/alizeb/.local/lib/python3.5/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """


In [27]:
%%time
clf = LogisticRegression()
clf.fit(train_vecs_cbowsg_sum, y_train1)

CPU times: user 11min 28s, sys: 380 ms, total: 11min 28s
Wall time: 11min 28s


In [28]:
import pickle
with open('../datasets/combined/models/classification/clf_lr_cbowsg_sum.p', 'wb') as fp:
    pickle.dump(clf, fp, protocol=pickle.HIGHEST_PROTOCOL)

In [31]:
import pickle
with open('../datasets/combined/models/classification/clf_lr_cbowsg_sum.p', 'rb') as fp:
    clf = pickle.load(fp)

In [32]:
clf.score(validation_vecs_cbowsg_sum, y_validation1)

0.986738499792789

## Separately trained Word2Vec with custom weighting (Average/Sum)

In [39]:
%%time
w2v_panama_hmean_01 = {}
for w in model_ug_cbow.wv.vocab.keys():
    if w in panama_hmean.keys():
        w2v_panama_hmean_01[w] = np.append(model_ug_cbow[w],model_ug_sg[w]) * panama_hmean[w]

/home/alizeb/.local/lib/python3.5/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  after removing the cwd from sys.path.


CPU times: user 4.92 s, sys: 24 ms, total: 4.94 s
Wall time: 4.91 s


In [46]:
%%time
train_vecs_w2v_panamahmean_mean_01 = scale(np.concatenate([get_w2v_general(z, 200, w2v_panama_hmean_01, 'mean') for z in x_train]))
validation_vecs_w2v_panamahmean_mean_01 = scale(np.concatenate([get_w2v_general(z, 200, w2v_panama_hmean_01, 'mean') for z in x_validation]))

CPU times: user 1min, sys: 1.43 s, total: 1min 1s
Wall time: 1min 1s


In [47]:
%%time
clf = LogisticRegression()
clf.fit(train_vecs_w2v_panamahmean_mean_01, y_train1)

CPU times: user 12min 18s, sys: 372 ms, total: 12min 19s
Wall time: 12min 19s


In [48]:
import pickle
with open('../datasets/combined/models/classification/clf_lr_w2v_panamahmean_mean.p', 'wb') as fp:
    pickle.dump(clf, fp, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
import pickle
with open('../datasets/combined/models/classification/clf_lr_w2v_panamahmean_mean.p', 'rb') as fp:
    clf = pickle.load(fp)

In [49]:
clf.score(validation_vecs_w2v_panamahmean_mean_01, y_validation1)

0.98487360132615

In [50]:
train_vecs_w2v_panamahmean_sum_01 = scale(np.concatenate([get_w2v_general(z, 200, w2v_panama_hmean_01, 'sum') for z in x_train]))
validation_vecs_w2v_panamahmean_sum_01 = scale(np.concatenate([get_w2v_general(z, 200, w2v_panama_hmean_01, 'sum') for z in x_validation]))

In [51]:
%%time
clf = LogisticRegression()
clf.fit(train_vecs_w2v_panamahmean_sum_01, y_train1)

CPU times: user 11min 24s, sys: 796 ms, total: 11min 25s
Wall time: 11min 35s


In [52]:
import pickle
with open('../datasets/combined/models/classification/clf_lr_w2v_panamahmean_sum_01.p', 'wb') as fp:
    pickle.dump(clf, fp, protocol=pickle.HIGHEST_PROTOCOL)

In [54]:
clf.score(validation_vecs_w2v_panamahmean_sum_01, y_validation1)

0.9859096560298384

# Neural Network with Word2Vec

In [55]:
train_w2v_final = train_vecs_w2v_panamahmean_mean_01
validation_w2v_final = validation_vecs_w2v_panamahmean_mean_01

In [62]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

filepath="../datasets/combined/models/classification/w2v_01_best_weights.{epoch:02d}-{val_acc:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
early_stop = EarlyStopping(monitor='val_acc', patience=5, mode='max') 
callbacks_list = [checkpoint, early_stop]
np.random.seed(seed)
model_w2v_01 = Sequential()
model_w2v_01.add(Dense(256, activation='relu', input_dim=200))
model_w2v_01.add(Dense(256, activation='relu'))
model_w2v_01.add(Dense(256, activation='relu'))
model_w2v_01.add(Dense(nb_classes, activation='sigmoid'))
model_w2v_01.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model_w2v_01.fit(train_w2v_final, y_train, validation_data=(validation_w2v_final, y_validation), 
                 epochs=100, batch_size=32, verbose=2, callbacks=callbacks_list)

Train on 945931 samples, validate on 9652 samples
Epoch 1/100
Epoch 00000: val_acc improved from -inf to 0.99529, saving model to ../datasets/combined/models/classification/w2v_01_best_weights.00-0.9953.hdf5
119s - loss: 0.0155 - acc: 0.9949 - val_loss: 0.0139 - val_acc: 0.9953
Epoch 2/100
Epoch 00001: val_acc improved from 0.99529 to 0.99629, saving model to ../datasets/combined/models/classification/w2v_01_best_weights.01-0.9963.hdf5
105s - loss: 0.0112 - acc: 0.9963 - val_loss: 0.0114 - val_acc: 0.9963
Epoch 3/100
Epoch 00002: val_acc improved from 0.99629 to 0.99655, saving model to ../datasets/combined/models/classification/w2v_01_best_weights.02-0.9965.hdf5
106s - loss: 0.0100 - acc: 0.9967 - val_loss: 0.0107 - val_acc: 0.9965
Epoch 4/100
Epoch 00003: val_acc did not improve
109s - loss: 0.0091 - acc: 0.9969 - val_loss: 0.0111 - val_acc: 0.9961
Epoch 5/100
Epoch 00004: val_acc did not improve
112s - loss: 0.0085 - acc: 0.9971 - val_loss: 0.0113 - val_acc: 0.9964
Epoch 6/100
Epoch

In [67]:
from keras.models import load_model
loaded_w2v_model = load_model('../datasets/combined/models/classification/w2v_01_best_weights.10-0.8048.hdf5')

In [68]:
loaded_w2v_model.evaluate(x=validation_w2v_final, y=y_validation)

15960/15960 [==============================] - 0s 20us/step


[0.4244666022615026, 0.8047619047619048]